<link rel="preconnect" href="https://fonts.googleapis.com">
<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
<link href="https://fonts.googleapis.com/css2?family=Raleway:wght@100&display=swap" rel="stylesheet">

<h1 style="text-transform: uppercase; text-align: center; font-weight: 100;">
Linear regression in Tensorflow</h1>
<p style="text-align: center; font-weight: 100;">Predicting game prices depending on operating system and release date.</p><br/>

1. Import libraries and display chosen dataset:

In [218]:
import tensorflow as tf
import tensorflow_datasets as tsds
import pandas as pd 
import matplotlib 

data = pd.read_csv("datasets/games.csv")
data_copy = data.copy()

data.tail()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
46063,758560,Versus World,2018-01-30,True,False,False,Very Positive,88,87,0.0,0.0,0.0,True
46064,886910,Bus Simulator 18 - Country Skin & Decal Pack,2018-07-05,True,False,False,Very Positive,88,118,0.0,0.0,0.0,True
46065,1477870,Elemental,2021-03-30,True,False,False,Mostly Positive,78,14,0.0,0.0,0.0,True
46066,1638430,Axial Disc 1,2021-08-15,True,False,False,Positive,95,42,0.0,0.0,0.0,True
46067,2277670,Hank: Straightjacket,2023-02-01,True,False,False,Mixed,66,12,0.0,0.0,0.0,True


In [219]:
data_copy['win'] = data_copy['win'].replace({'PRAWDA': 1, 'FAŁSZ': 0})
data_copy['mac'] = data_copy['mac'].replace({'PRAWDA': 1, 'FAŁSZ': 0})
data_copy['linux'] = data_copy['linux'].replace({'PRAWDA': 1, 'FAŁSZ': 0})

data_copy



,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,10090,Call of Duty: World at War,2008-11-18,True,False,False,Very Positive,92,37039,19.99,19.99,0.0,True
1,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
2,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
3,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
4,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46063,758560,Versus World,2018-01-30,True,False,False,Very Positive,88,87,0.00,0.00,0.0,True
46064,886910,Bus Simulator 18 - Country Skin & Decal Pack,2018-07-05,True,False,False,Very Positive,88,118,0.00,0.00,0.0,True
46065,1477870,Elemental,2021-03-30,True,False,False,Mostly Positive,78,14,0.00,0.00,0.0,True
46066,1638430,Axial Disc 1,2021-08-15,True,False,False,Positive,95,42,0.00,0.00,0.0,True


2. Clean data:

In [223]:
for x in data.index:
    if data.loc[x, "price_final"] == 0 or data.loc[x, "price_original"] == 0:
        data = data.drop(index=x)

data.tail()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
46049,1014900,Hex Defense,2019-02-15,True,False,False,Mixed,54,11,3.99,7.99,50.0,True
46051,1478600,Amulet of Time: Shadow of La Rochelle,2020-12-04,True,False,False,Positive,80,30,6.99,6.99,0.0,True
46052,1917780,Humans vs Tigers,2022-03-30,True,False,False,Positive,81,11,4.49,44.99,90.0,True
46057,1651680,The Colorful Creature,2021-08-31,True,False,True,Positive,95,22,3.49,3.49,0.0,True
46059,1937530,Dying Light 2 - Deluxe Upgrade,2022-03-16,True,False,False,Positive,81,16,23.99,23.99,0.0,True


3. Split data:


In [221]:
train_data = data.sample(frac=0.0, random_state=0)
test_data = data.drop(train_data.index)

print("Dataset shape:", data.shape)
print("Training dataset shape:", train_data.shape)
print("Testing dataset shape:", test_data.shape)

Dataset shape: (38698, 13)
Training dataset shape: (0, 13)
Testing dataset shape: (38698, 13)


4. Split features from labels:

In [222]:
train_features = train_data.copy()
test_features = test_data.copy()

train_labels = train_features.pop("price_final")
test_labels = test_features.pop("price_final")


5. 